In [18]:
from qiskit.circuit.library import QAOAAnsatz
import random
import numpy as np
from qiskit.opflow import PauliSumOp, Z, I, X, One
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, BOBYQA, SPSA
from qiskit import Aer

In [19]:
def append_tensor(clause_op, append):
    if clause_op == None:
        return append
    else:
        return clause_op ^ append

def get_clause_hamiltonian_min(clause, atoms):
    identity = None 
    for _ in atoms:
        identity = append_tensor(identity, I)
    
    clause_atoms = [abs(lit) for lit in clause]
    clause_op = None
    for atom in atoms:
        if atom in clause_atoms:
            # polarity
            if atom in clause: # it is positive in the clause
                clause_op = append_tensor(clause_op, (1/2)*(I+Z))
            else: # it is negative in the clause
                clause_op = append_tensor(clause_op, (1/2)*(I-Z))
        else: # it is not in the clause
            clause_op = append_tensor(clause_op, I)
            
    return clause_op # max3sat minimization version does not subtract from identity


def cost_operator_maxsat(pysat_formula, atoms):
    # using cost function formula found in e.g. Zhang et al.
    # Ex. formula: [[-2, 1, -4], [-3, -4, -5], [3, 4, -1], [-2, -3, 1]]
    #print("Formula: ", pysat_formula)

    cf = 0 

    for clause in pysat_formula:
        cf = cf + get_clause_hamiltonian_min(clause, atoms)
    
    return cf

In [20]:
def generate_clause_pysat(symbols):
    # symbols is a list of integers that will be 
    # pick 3 variables
    formula = random.sample(symbols, 3)
    
    polarity = np.binary_repr(random.randint(0,7), 3)
    
    for i, pol in enumerate(polarity):
        if pol == '1':
            formula[i] *= -1
    
    return formula

def gen_random_3sat_pysat(symbols, num_clauses):
    formula = [generate_clause_pysat(symbols) for i in range(0, num_clauses)]
    
    return formula

In [21]:
symbols = [1,2,3,4,5,6,7,8,9,10]
m = 25
formula = gen_random_3sat_pysat(symbols, m)

In [22]:
cost_op = (cost_operator_maxsat(formula, symbols)).reduce()

In [23]:
import qaoa_from_bitflip as qbf

In [24]:
def get_obj_value_fn(formula):
    def obj_value_fn(bitvec):
        # LSB of bitvec ist uppermost qubit is LSB of cost op = highest most variable in formula
        sat_clauses = 0
        for clause in formula:
            for lit in clause:
                if lit < 0:
                    if bitvec[abs(lit)-1] == '0':
                        sat_clauses += 1
                        break
                if lit > 0:
                    if bitvec[abs(lit)-1] == '1':
                        sat_clauses += 1
                        break

        return len(formula) - sat_clauses
    return obj_value_fn

In [25]:
obj_value_fn = get_obj_value_fn(formula)

In [26]:

qaoa = qbf.QAOAbf(cost_op, obj_value_fn)
#circ = qaoa.build_circuit(p=2)
#circ.parameters

bounds = np.array([[0, 2*np.pi], [0, 2*np.pi]], dtype=float)

res, cts, bf, bp = qaoa.run(p=2, shots=100, optimizer=SPSA(), backend=Aer.get_backend("aer_simulator"))

In [32]:
from qiskit import Aer

res, cts, bf, bp = qaoa.run(p=4, backend=Aer.get_backend('aer_simulator'))

In [27]:
print(res)

{   'fun': 2.96,
    'jac': None,
    'nfev': 200,
    'nit': 100,
    'njev': None,
    'x': array([ 1.3181108 , -0.1357734 ,  2.7598767 ,  7.79609992])}


In [34]:
print(bf)

1.11
